In [48]:
# coding: utf-8
import pandas as pd
import numpy as np
import data_load as dl

In [55]:
dl.get_data("""
with
random_rate_old as 
(
    select 
    user_id, avg(result) as random_value, count(distinct insert_time) as attempts
    from tl.game_results
    group by user_id
), 
left_tbl as 
(
    select battle1 as item, user_id, "user"
        , case 
            when result = 0 then 1
            else 0
        end as is_selected
            --, row_number() as id
    from tl.game_results
),
right_tbl as 
(
        select battle2 as item, user_id, "user"
                    , case 
                        when result = 1 then 1
                        else 0
                    end as is_selected
    from tl.game_results
),
pre_items as 
(
    select * from left_tbl

    union all

    select * from right_tbl
),
items as 
(
    select *, row_number() over() as id
    from pre_items
),
global_stat as 
(
    select item
                , sum(is_selected) as selected
                , count(is_selected) as total
                , sum(is_selected)::float / count(is_selected) as select_rate
    from items
    group by item
),
personal_stat as 
(
    select item, user_id
                , sum(is_selected) as selected
                , count(is_selected) as total
                , sum(is_selected)::float / count(is_selected) as select_rate
    from items
    group by item, user_id
),
indiv_stat as 
(
    select ps.*
            , gs.select_rate as global_select_rate
            , case 
                when (ps.select_rate <= 0.05) or (ps.select_rate >= 0.95) then 0
                else  round((100 * abs(gs.select_rate - ps.select_rate))::decimal, 0)
              end as original_score
    from personal_stat ps
    left outer join global_stat gs on 
        (ps.item=gs.item)
),
original as 
(
    select user_id, sum(original_score) as original_score, sum(total) as attemps
    from  indiv_stat
    group by user_id
),
random_rate as 
(
    select user_id
            , stddev_pop(select_rate)
            ,  case 
                when stddev_pop(select_rate) <= 0.1 then 0
                when stddev_pop(select_rate) >= 0.9 then 100
                else round((100*(stddev_pop(select_rate)))::decimal,0)
            end as non_random_rate
    from personal_stat
    group by user_id
)

select o.user_id, o.attemps, o.original_score, non_random_rate
from original o
left outer join random_rate rr on
    (rr.user_id = o.user_id)


""")

,user_id,attemps,original_score,non_random_rate
0,435209913,6,34,41
1,249792088,124,140,34
2,402555773,18,40,40
3,299230584,50,145,31


In [56]:
dl.get_data("""
select *
from tl.game_results
where "user"='noramornomore'

--group by 1,2


""")

,battle1,battle2,result,user,user_id,insert_time
0,Незнайка на Луне,Властелин Колец,1,noramornomore,435209913,2023-02-21 18:27:38.744460
1,Мастер и Маргарита,Властелин Колец,1,noramornomore,435209913,2023-02-21 18:27:46.871958
2,Айвенго,Мастер и Маргарита,1,noramornomore,435209913,2023-02-21 18:27:53.644013
